In [5]:
import cv2 as cv
import numpy as np
import os
from scipy import ndimage
img= cv.imread('BMVP.png')
hsv= cv.cvtColor(img,cv.COLOR_BGR2HSV)
lower_p = np.array([0,0,150])
upper_p =np.array([255,255,255])
mask=cv.inRange(hsv, lower_p, upper_p) # background noise removed.
res= cv.bitwise_and(hsv,img,mask=mask) #NOT USEFUL
kernel = np.ones((5,5), np.uint8)  # erosion
img_erosion = cv.erode(mask, kernel, iterations=1) 
kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])]) # finding edges which may be useful in segmentation ,Can use some other technique also 
out_l = ndimage.convolve(img_erosion, kernel_laplace, mode='reflect')

cv.imshow('Erosion', img_erosion) 
cv.imshow('img',img)
cv.imshow('mask',mask)
# cv.imshow('res',res)
cv.imshow('edge', out_l)  # this one may not be compulsary if we do segmentation by some other method.
cv.waitKey(0)
cv.destroyAllWindows()


In [18]:
for f in os.listdir("train"):
    input_image = f
    output_image = input_image
    img= cv.imread('train/'+input_image)
    hsv= cv.cvtColor(img,cv.COLOR_BGR2HSV)
    lower_p = np.array([0,0,150])
    upper_p =np.array([255,255,255])
    mask=cv.inRange(hsv, lower_p, upper_p)
    cv.imwrite('train_out/'+output_image,mask)

In [1]:
import os
import random
from shutil import copyfile

def img_train_test_split(img_source_dir, train_size):
    """
    Randomly splits images over a train and validation folder, while preserving the folder structure
    
    Parameters
    ----------
    img_source_dir : string
        Path to the folder with the images to be split. Can be absolute or relative path   
        
    train_size : float
        Proportion of the original images that need to be copied in the subdirectory in the train folder
    """    
    if not (isinstance(img_source_dir, str)):
        raise AttributeError('img_source_dir must be a string')
        
    if not os.path.exists(img_source_dir):
        raise OSError('img_source_dir does not exist')
        
    if not (isinstance(train_size, float)):
        raise AttributeError('train_size must be a float')
        
    # Set up empty folder structure if not exists
    if not os.path.exists('data'):
        os.makedirs('data')
    else:
        if not os.path.exists('data/train'):
            os.makedirs('data/train')
        if not os.path.exists('data/validation'):
            os.makedirs('data/validation')
            
    # Get the subdirectories in the main image folder
#     subdirs = [subdir for subdir in os.listdir(img_source_dir) if os.path.isdir(os.path.join(img_source_dir, subdir))]

#     for subdir in subdirs:
#         subdir_fullpath = os.path.join(img_source_dir, subdir)
#         if len(os.listdir(subdir_fullpath)) == 0:
#             print(subdir_fullpath + ' is empty')
#             break

#         train_subdir = os.path.join('data/train', subdir)
#         validation_subdir = os.path.join('data/validation', subdir)

#         # Create subdirectories in train and validation folders
#         if not os.path.exists(train_subdir):
#             os.makedirs(train_subdir)

#         if not os.path.exists(validation_subdir):
#             os.makedirs(validation_subdir)

    train_counter = 0
    validation_counter = 0
    subdir_fullpath = img_source_dir
    train_subdir = "data/train"
    validation_subdir = "data/validation"
    # Randomly assign an image to train or validation folder
    for filename in os.listdir(subdir_fullpath):
        if filename.endswith(".jpg") or filename.endswith(".png"): 
            fileparts = filename.split('.')

            if random.uniform(0, 1) <= train_size:
                copyfile(os.path.join(subdir_fullpath, filename), os.path.join(train_subdir, fileparts[0] + '.' + fileparts[1]))
                train_counter += 1
            else:
                copyfile(os.path.join(subdir_fullpath, filename), os.path.join(validation_subdir, fileparts[0] + '.' + fileparts[1]))
                validation_counter += 1
                    
#         print('Copied ' + str(train_counter) + ' images to data/train/' + subdir)
#         print('Copied ' + str(validation_counter) + ' images to data/validation/' + subdir)

In [2]:
img_train_test_split("train_out", 0.9)